In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from vecstack import stacking
#import for the lstm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [60]:
#Reading the data from theonion vs nottheonion
df = pd.read_csv('C:/Users/ASUS/Desktop/twitter-Fake-News-Detection-master/twitter-Fake-News-Detection-master/FinalDS_AdditionalFeatures.csv')
df.sample(5)
df=df[pd.to_numeric(df['FinalLabel'], errors='coerce').notnull()]
df=df[df.FinalLabel.apply(lambda x: x.isnumeric())]


In [61]:
#splitting test-train-x-y-Features
y = df[['FinalLabel']]
X = df.iloc[:,[2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 21, 22, 23, 24, 26, 27, 28, 29]]
Xlstm=df.iloc[:,25]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
Xlstm_train, Xlstm_test, y_train, y_test = train_test_split(Xlstm, y, test_size=0.2, random_state=0)

In [62]:
y_train

,FinalLabel
228,0
66,1
160,0
109,0
99,0
116,0
34,0
184,0
165,0
164,0


In [63]:
#converting text to sequences could be later optimized with word2vec or something
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(Xlstm_train)
sequences = tok.texts_to_sequences(Xlstm_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [64]:
#Defining our network 
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [65]:
#RMSprop is op asf
modeldl = RNN()
modeldl.summary()
modeldl.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_9 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [66]:
#meh
modeldl.fit(sequences_matrix,y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 154 samples, validate on 39 samples
Epoch 1/10
154/154 [==============================] - 2s 14ms/step - loss: 0.6924 - acc: 0.5260 - val_loss: 0.6872 - val_acc: 0.5897
Epoch 2/10
154/154 [==============================] - 0s 3ms/step - loss: 0.6820 - acc: 0.6234 - val_loss: 0.6765 - val_acc: 0.5897
Epoch 3/10
154/154 [==============================] - 0s 2ms/step - loss: 0.6715 - acc: 0.6234 - val_loss: 0.6754 - val_acc: 0.5897
Epoch 4/10
154/154 [==============================] - 0s 2ms/step - loss: 0.6562 - acc: 0.6234 - val_loss: 0.6652 - val_acc: 0.5897
Epoch 5/10
154/154 [==============================] - 0s 3ms/step - loss: 0.6329 - acc: 0.6234 - val_loss: 0.6544 - val_acc: 0.5897
Epoch 6/10
154/154 [==============================] - 0s 3ms/step - loss: 0.5956 - acc: 0.6234 - val_loss: 0.6493 - val_acc: 0.5641
Epoch 7/10
154/154 [==============================] - 0s 2ms/step - loss: 0.5810 - acc: 0.7208 - val_loss: 0.6331 - val_acc: 0.5641
Epoch 8/10
154/154 [==========